In [19]:
from tensorflow import keras
import numpy as np
import os
import shutil
from keras.callbacks import EarlyStopping

In [20]:
if os.path.exists('./test'):
    pass
#         shutil.rmtree('./test')
#         shutil.rmtree('./train')
        
if not os.path.exists('./test'):
    root= '../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset' #path of the original folder  
    classes=[]
    for i in os.listdir(root):
        if os.path.isdir(root+'/'+i):
            classes.append(i)
    print(classes)
    destination='/kaggle/working/'

    for i in classes:
        if not os.path.exists(destination +'/train/' + i):
            os.makedirs(destination +'/train/' + i)
        if not os.path.exists(destination +'/test/' + i):
            os.makedirs(destination +'/test/' + i)

        source = root + '/' + i + '/' +i
        allFileNames = os.listdir(source)

        np.random.shuffle(allFileNames)

        test_ratio = 0.20

        train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - test_ratio))])

        train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
        test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]

        for name in train_FileNames:
          shutil.copy(name, destination +'/train/' + i)

        for name in test_FileNames:
          shutil.copy(name, destination +'/test/' + i)

In [21]:
import tensorflow
img_size = (300, 300)
batch_size = 8
train=tensorflow.keras.preprocessing.image_dataset_from_directory(
        './train',
        batch_size=batch_size,
        image_size=img_size,  
        shuffle=True,
        seed=1,
        validation_split=0.1,
        subset="training",
)

val=tensorflow.keras.preprocessing.image_dataset_from_directory(
        './train',
        batch_size=batch_size,
        image_size=img_size,  
        shuffle=True,
        seed=1,
        validation_split=0.1,
        subset="validation",
)

test=tensorflow.keras.preprocessing.image_dataset_from_directory(
        './test',
        batch_size=batch_size,
        image_size=img_size,  
        shuffle=True,
        seed=1,
        validation_split=None
)

Found 7200 files belonging to 9 classes.
Using 6480 files for training.
Found 7200 files belonging to 9 classes.
Using 720 files for validation.
Found 1800 files belonging to 9 classes.


In [22]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D, MaxPool2D, Flatten, Dropout

In [23]:
model=Sequential([
    tensorflow.keras.layers.experimental.preprocessing.Rescaling(1./255),
    Conv2D(64, kernel_size=(3,3), input_shape=(300, 300,3), activation='relu'),
    MaxPool2D(pool_size=(2,2), padding='same'),
    Dropout(0.2),
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2,2), padding='same'),
    Dropout(0.3),
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2,2), padding='same'),
    Dropout(0.3),
    Conv2D(256, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2,2), padding='same'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(9,activation='softmax')
])

In [24]:
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=3, min_delta=0.01)

In [25]:
model.compile(optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"])


In [26]:
model.fit(train, validation_data=val, batch_size=8, epochs=10, callbacks=[es])

Epoch 1/10
810/810 [==============================] - 56s 68ms/step - loss: 1.8029 - accuracy: 0.3307 - val_loss: 1.1506 - val_accuracy: 0.6333
Epoch 2/10
810/810 [==============================] - 55s 67ms/step - loss: 0.9034 - accuracy: 0.6843 - val_loss: 0.5673 - val_accuracy: 0.8097
Epoch 3/10
810/810 [==============================] - 55s 67ms/step - loss: 0.5705 - accuracy: 0.7998 - val_loss: 0.4004 - val_accuracy: 0.8486
Epoch 4/10
810/810 [==============================] - 54s 66ms/step - loss: 0.3924 - accuracy: 0.8619 - val_loss: 0.3024 - val_accuracy: 0.8931
Epoch 5/10
810/810 [==============================] - 54s 66ms/step - loss: 0.2889 - accuracy: 0.9009 - val_loss: 0.2700 - val_accuracy: 0.9194
Epoch 6/10
810/810 [==============================] - 54s 65ms/step - loss: 0.2350 - accuracy: 0.9184 - val_loss: 0.2288 - val_accuracy: 0.9153
Epoch 7/10
810/810 [==============================] - 54s 67ms/step - loss: 0.1880 - accuracy: 0.9329 - val_loss: 0.2493 - val_accuracy:

In [27]:
results = model.evaluate(test)
print("test loss, test acc:", results)

225/225 [==============================] - 12s 51ms/step - loss: 0.1981 - accuracy: 0.9444
test loss, test acc: [0.19810649752616882, 0.9444444179534912]


In [39]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input,decode_predictions

In [40]:
model_res =Xception(input_shape=(300,300,3),include_top=False,weights='imagenet')

In [41]:
inputs = tf.keras.Input(shape=(300,300,3))
x1 = preprocess_input(inputs)
x1 = model_res(x1, training=False)
x2 = tf.keras.layers.Dropout(0.3)(x1)
# x3 = tf.keras.layers.GlobalAveragePooling2D()(x1)
x3 = tf.keras.layers.Flatten()(x2)
x6 =tf.keras.layers.Dropout(0.3)(x3)
outputs3 = tf.keras.layers.Dense(10)(x6)
model1 = tf.keras.Model(inputs=inputs, outputs=outputs3)

In [42]:
base_learning_rate = 0.00001
model1.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [43]:
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=3, min_delta=0.01)
model1.fit(train,epochs=10,validation_data=val, callbacks=[es])

Epoch 1/10
810/810 [==============================] - 191s 232ms/step - loss: 0.2491 - accuracy: 0.9205 - val_loss: 0.0064 - val_accuracy: 0.9972
Epoch 2/10
810/810 [==============================] - 187s 230ms/step - loss: 0.0081 - accuracy: 0.9977 - val_loss: 9.0559e-04 - val_accuracy: 1.0000
Epoch 3/10
810/810 [==============================] - 186s 229ms/step - loss: 0.0062 - accuracy: 0.9980 - val_loss: 0.0221 - val_accuracy: 0.9944
Epoch 00003: early stopping


In [44]:
results = model1.evaluate(test)
print("test loss, test acc:", results)

225/225 [==============================] - 14s 60ms/step - loss: 0.0081 - accuracy: 0.9978
test loss, test acc: [0.008115394972264767, 0.9977777600288391]
